**SRINIVAS srinivasacademics@gmail.com**

**IMPORT AND SAVE RAW DATA IN DATABASE**

**DATA CHOOSEN - SMALL**

**DATASTORED FORMAT CSV FILE**

**DATA STORED IN DATABASE SQlite3 AND FETCHED**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import sqlite3
import os
print("path:", os.getcwd())

path: /content


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tweets.csv", encoding="utf-8")

In [ ]:
# Check structure
print(df.shape)   # number of rows, columns
print(df.head())  # first 5 rows

(52542, 10)
      author                                            content country  \
0  katyperry  Is history repeating itself...?#DONTNORMALIZEH...     NaN   
1  katyperry  @barackobama Thank you for your incredible gra...     NaN   
2  katyperry                Life goals. https://t.co/XIn1qKMKQl     NaN   
3  katyperry            Me right now 🙏🏻 https://t.co/gW55C1wrwd     NaN   
4  katyperry  SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...     NaN   

          date_time            id language  latitude  longitude  \
0  12/01/2017 19:52  8.196330e+17       en       NaN        NaN   
1  11/01/2017 08:38  8.191010e+17       en       NaN        NaN   
2  11/01/2017 02:52  8.190140e+17       en       NaN        NaN   
3  11/01/2017 02:44  8.190120e+17       en       NaN        NaN   
4  10/01/2017 05:22  8.186890e+17       en       NaN        NaN   

   number_of_likes  number_of_shares  
0             7900              3472  
1             3689              1380  
2            1034

In [ ]:
# Create (or connect to) database file
conn = sqlite3.connect("tweets.db")

# Create cursor for SQL queries
cursor = conn.cursor()


In [ ]:
# Save dataframe into a new table called 'tweets'
df.to_sql("tweets", conn, if_exists="replace", index=False)

print("CSV file stored successfully into SQLite3 database!")


CSV file stored successfully into SQLite3 database!


In [ ]:
# Count rows in table
cursor.execute("SELECT COUNT(*) FROM tweets")
print("Total rows stored:", cursor.fetchone()[0])

# Fetch few rows
cursor.execute("SELECT * FROM tweets LIMIT 5")
rows = cursor.fetchall()

for row in rows:
    print(row)

Total rows stored: 52542
('katyperry', 'Is history repeating itself...?#DONTNORMALIZEHATE https://t.co/ngG11quhmK', None, '12/01/2017 19:52', 8.19633e+17, 'en', None, None, 7900, 3472)
('katyperry', '@barackobama Thank you for your incredible grace in leadership and for being an exceptional… https://t.co/ZuQLZpt6df', None, '11/01/2017 08:38', 8.19101e+17, 'en', None, None, 3689, 1380)
('katyperry', 'Life goals. https://t.co/XIn1qKMKQl', None, '11/01/2017 02:52', 8.19014e+17, 'en', None, None, 10341, 2387)
('katyperry', 'Me right now 🙏🏻 https://t.co/gW55C1wrwd', None, '11/01/2017 02:44', 8.19012e+17, 'en', None, None, 10774, 2458)
('katyperry', "SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ https://t.co/0shuUYUBEv", None, '10/01/2017 05:22', 8.18689e+17, 'en', None, None, 17620, 4655)


In [ ]:
cursor.execute("PRAGMA table_info(tweets);")
table_info = cursor.fetchall()

df = pd.DataFrame(table_info, columns=["cid", "name", "type", "notnull", "dflt_value", "pk"])
print(df)

   cid              name     type  notnull dflt_value  pk
0    0            author     TEXT        0       None   0
1    1           content     TEXT        0       None   0
2    2           country     TEXT        0       None   0
3    3         date_time     TEXT        0       None   0
4    4                id     REAL        0       None   0
5    5          language     TEXT        0       None   0
6    6          latitude     REAL        0       None   0
7    7         longitude     REAL        0       None   0
8    8   number_of_likes  INTEGER        0       None   0
9    9  number_of_shares  INTEGER        0       None   0


**TEXT CLEANING**

**REMOVING UNNECESSARY TEXT AND EMOJIS**

**NORMALIZATION LOWERCASING, TOKENIZING, LEMMATIZING, POS TAGGING**

In [ ]:
import sqlite3
import pandas as pd
import spacy

In [ ]:
# 1. Connect to your SQLite3 database
conn = sqlite3.connect("tweets.db")

In [ ]:
# 2. Load data from the tweets table into Pandas
df = pd.read_sql_query("SELECT * FROM tweets", conn)

In [ ]:
# check first few rows
print(df.head(2))

      author                                            content country  \
0  katyperry  Is history repeating itself...?#DONTNORMALIZEH...    None   
1  katyperry  @barackobama Thank you for your incredible gra...    None   

          date_time            id language  latitude  longitude  \
0  12/01/2017 19:52  8.196330e+17       en       NaN        NaN   
1  11/01/2017 08:38  8.191010e+17       en       NaN        NaN   

   number_of_likes  number_of_shares  
0             7900              3472  
1             3689              1380  


In [ ]:
# 3. Text Preprocessing with Pandas
# make lowercase
df['content_clean'] = df['content'].str.lower()
df.head(2)

,author,content,country,date_time,id,language,latitude,longitude,number_of_likes,number_of_shares,content_clean
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,None,12/01/2017 19:52,8.196330e+17,en,NaN,NaN,7900,3472,is history repeating itself...?#dontnormalizeh...
1,katyperry,@barackobama Thank you for your incredible gra...,None,11/01/2017 08:38,8.191010e+17,en,NaN,NaN,3689,1380,@barackobama thank you for your incredible gra...


In [ ]:
# remove \xa0, ?, # characters
# remove punctuation
df['content_clean'] = df['content_clean'].str.replace('\xa0', ' ')
df['content_clean'] = df['content_clean'].str.replace('?', ' ')
df['content_clean'] = df['content_clean'].str.replace('#', ' ')
df['content_clean'] = df['content_clean'].str.replace(r'[^\w\s]', '', regex=True)
print(df[['content', 'content_clean']].head(2))


                                             content  \
0  Is history repeating itself...?#DONTNORMALIZEH...   
1  @barackobama Thank you for your incredible gra...   

                                       content_clean  
0  is history repeating itself  dontnormalizehate...  
1  barackobama thank you for your incredible grac...  


In [ ]:
print(df[['content', 'content_clean']].head(5))

                                             content  \
0  Is history repeating itself...?#DONTNORMALIZEH...   
1  @barackobama Thank you for your incredible gra...   
2                Life goals. https://t.co/XIn1qKMKQl   
3            Me right now 🙏🏻 https://t.co/gW55C1wrwd   
4  SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...   

                                       content_clean  
0  is history repeating itself  dontnormalizehate...  
1  barackobama thank you for your incredible grac...  
2                      life goals httpstcoxin1qkmkql  
3                   me right now  httpstcogw55c1wrwd  
4  sisters are doin it for themselves  httpstco0s...  


In [ ]:
# look at a single text string
desc = df.content_clean.loc[0]
desc

'is history repeating itself  dontnormalizehate httpstcongg11quhmk'

In [ ]:
# 4. Text Preprocessing with spaCy
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])  # disable heavy components

In [ ]:
# turn the description into a spacy object
doc = nlp(desc)
doc

is history repeating itself  dontnormalizehate httpstcongg11quhmk

In [ ]:
# tokenize, lemmatize and remove stop words
[token.lemma_ for token in doc if not token.is_stop][:10]

['history', 'repeat', ' ', 'dontnormalizehate', 'httpstcongg11quhmk']

In [ ]:
# turn the list into a string
' '.join([token.lemma_ for token in doc if not token.is_stop])

'history repeat   dontnormalizehate httpstcongg11quhmk'

In [ ]:
# function to tokenize, lemmatize, remove stop words
from spacy.lang.en.stop_words import STOP_WORDS
def token_lemma_nonstop(text):
    doc = nlp(text.lower())  # lowercase and tokenize
    return " ".join([token.lemma_ for token in doc if token.is_alpha and token.text not in STOP_WORDS])
print(df[['content', 'content_clean']].head(5))

                                             content  \
0  Is history repeating itself...?#DONTNORMALIZEH...   
1  @barackobama Thank you for your incredible gra...   
2                Life goals. https://t.co/XIn1qKMKQl   
3            Me right now 🙏🏻 https://t.co/gW55C1wrwd   
4  SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...   

                                       content_clean  
0  is history repeating itself  dontnormalizehate...  
1  barackobama thank you for your incredible grac...  
2                      life goals httpstcoxin1qkmkql  
3                   me right now  httpstcogw55c1wrwd  
4  sisters are doin it for themselves  httpstco0s...  


In [ ]:
# apply to cleaned tweets
df['content_clean'] = df['content_clean'].apply(token_lemma_nonstop)

print(df[['content', 'content_clean']].head(5))


                                             content  \
0  Is history repeating itself...?#DONTNORMALIZEH...   
1  @barackobama Thank you for your incredible gra...   
2                Life goals. https://t.co/XIn1qKMKQl   
3            Me right now 🙏🏻 https://t.co/gW55C1wrwd   
4  SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...   

                                       content_clean  
0                   history repeat dontnormalizehate  
1  barackobama thank incredible grace leadership ...  
2                                          life goal  
3                                              right  
4                                        sister doin  


**COUNTING WORDS USING MULTIPROCESSING**

In [ ]:
from collections import Counter
from multiprocessing import Pool, cpu_count

In [ ]:
# Function to count words in a chunk of texts
def count_words_in_chunk(texts):
    counter = Counter()
    for text in texts:
        words = text.split()
        counter.update(words)
    return counter

In [ ]:
# Function to split list into n chunks
def chunks(lst, n):
    k, m = divmod(len(lst), n)
    return [lst[i*k + min(i, m):(i+1)*k + min(i+1, m)] for i in range(n)]

In [ ]:
if __name__ == "__main__":
    texts = df['content_clean'].tolist()  # your cleaned texts
    num_cores = cpu_count()  # number of CPU cores
    text_chunks = chunks(texts, num_cores)  # split data into chunks

    with Pool(num_cores) as pool:
        results = pool.map(count_words_in_chunk, text_chunks)

    # Combine results from all processes
    total_counts = Counter()
    for r in results:
        total_counts.update(r)

    # Show the 20 most common words
    top_words = total_counts.most_common(20)

    print("Top 20 words and their counts:\n")
    for word, count in top_words:
      print(f"{word:15} : {count}")

Top 20 words and their counts:

love            : 3880
thank           : 3796
not             : 3047
m               : 2576
new             : 2462
amp             : 2438
tonight         : 2433
happy           : 1897
good            : 1893
day             : 1862
s               : 1710
today           : 1591
time            : 1534
u               : 1519
get             : 1511
watch           : 1445
like            : 1440
come            : 1373
president       : 1357
de              : 1253
